### #MeToo Twitter Sentiment Analysis

#### Name: Amrishan Narasingaperumal Vairavan

In [2]:
import json
import pandas as pd
import tweepy
import string
import nltk
import re
from textblob import TextBlob

In [3]:
st = nltk.corpus.stopwords.words
lemma = nltk.stem.wordnet.WordNetLemmatizer()

### Importing states,abbrevations and postal code

In [4]:
states = pd.read_csv('usa_states.csv')
states_lst = states.values.tolist()
state_acr = states.iloc[:,0].values.tolist()

### Converting from json format to list

In [5]:
tweets_data = []
tweets_file = open('tweets.json', "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [6]:
df = pd.DataFrame(tweets_data)

### unpacking locations data from user 

In [7]:
loca = []
for i in df.index:
    loca.append(df['user'][i]['location'])

In [8]:
location = pd.DataFrame(loca, columns = ['location'])
act_location = pd.DataFrame(loca, columns = ['act_location'])

In [9]:
df_loca = df.join(location, how = 'inner')
df_location = df_loca.join(act_location, how = 'inner')

In [10]:
df_state = df_location.fillna('nil')

### Checking location for states and replacing them with acronyms of the state

In [11]:
for state in states_lst:
    a = "{}|{}".format(state[0],state[1])
    st_list = df_state['location'][df_state['location'].str.contains(a, na = False)].values.tolist()
    df_state['location'][df_state['location'].str.contains(a, na = False)] = state[0]

C:\Users\amrishan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
def rep_unknown(mess):
    if mess in state_acr:
        return(mess)
    elif mess == "nil":
        return('nil')
    else:
        return("unknown")

In [13]:
df1 = df_state['location'].apply(rep_unknown)

In [14]:
df_state.drop(['location'], axis = 1,inplace = True)

In [15]:
df_state_text = df_state.join(df1,how = "inner")

### Grouping texts by state

In [16]:
state_grouped = df_state_text.groupby('location')['text'].apply(lambda x: x.sum())

### Text preprocessing

In [17]:
def clean(mess):
    message = str(mess)
    sent = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([0-9])","",message).split())
    no_punc = [char for char in sent if char not in string.punctuation]
    no_punc_str = "".join(no_punc)
    no_stop = [ word.lower() for word in no_punc_str.split() if word.strip().lower() not in st('english')]
    normalized = list(map(lambda x: lemma.lemmatize(x),no_stop))
    sentence = " ".join(normalized)
    mess = sentence.replace('rt','')
    return(mess)

In [18]:
 def clean_sent(message):
    mess = str(message)
    sent = ' '.join(re.sub("(S[A-Za-z\(\=]+)|(s[A-Za-z\=]+)|([\)]+|([,]))","",mess).split())
    return(sent)

In [19]:
df_grouped = pd.DataFrame(state_grouped,columns = ['text'])

In [20]:
df_sent = df_grouped['text'].apply(clean)

### sentiment analysis

In [21]:
 def text(sent):
    file = TextBlob(sent)
    return(file.sentiment) 

In [24]:
sentiment_s = df_sent.apply(text)
df_sentiment = pd.DataFrame(sentiment_s)
df_sent_clean = df_sentiment.assign(text = df_sentiment.text.apply(clean_sent))
df_split = pd.DataFrame(df_sent_clean.text.str.split(' ',1).tolist(), columns = ['polarity','subjectivity'])
df_index = pd.DataFrame(df_grouped.index.tolist())
df_split_index = df_split.join(df_index,how = "inner")
df_split_index.to_csv('sentiment_analysis_metoo5.csv')